
<a href="https://colab.research.google.com/github/aviadr1/learn-advanced-python/blob/master/content/08_test_driven_development/08-unittest_mock.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" 
     title="Open this file in Google Colab" alt="Colab"/>
</a>


# What Is Mocking?

A mock object substitutes and imitates a real object within a testing environment.

For example, if your code makes HTTP requests to external services, then your tests execute predictably only so far as the services are behaving as you expected. Sometimes, a temporary change in the behavior of these external services can cause intermittent failures within your test suite.

Because of this, it would be better for you to test your code in a __controlled environment__. Replacing the actual request with a mock object would allow you to __simulate__ external service outages and successful responses in a predictable way.

Sometimes, it is difficult to test certain areas of your codebase. Such areas include except blocks and if statements that are hard to satisfy. Using Python mock objects can help you __control the execution path of your code__ to reach these areas and improve your code coverage.

Another reason to use mock objects is to better understand how you’re using their real counterparts in your code. A Python mock object contains data about its usage that you can inspect such as:

- If you called a method
- How you called the method
- How often you called the method

Understanding what a mock object does is the first step to learning how to use one.

Now, you’ll see how to use Python mock objects.

# The Python Mock Library
We will be using `unittest.mock` 

> Note: The standard library includes unittest.mock in Python 3.3 and later. If you’re using an older version of Python, you’ll need to install the official backport of the library. To do so, install mock from PyPI:

```
$ pip install mock
```

`unittest.mock` provides a class called `Mock` which you will use to imitate real objects in your codebase. Mock offers incredible flexibility and insightful data. 

The library also provides a function, called `patch()`, which replaces the real objects in your code with Mock instances. You can use `patch()` as either a decorator or a context manager, giving you control over the scope in which the object will be mocked. Once the designated scope exits, `patch()` will clean up your code by replacing the mocked objects with their original counterparts.

Let’s dive in

In [19]:
from unittest.mock import MagicMock as Mock

mock = Mock()
mock

<MagicMock id='140629024763016'>

Now, you are able to substitute an object in your code with your new Mock. You can do this by passing it as an argument to a function or by redefining another object:

In [0]:
def do_something(duck):
    duck.walk()
    duck.talk()
    return duck.feed("bread")

# Pass mock as an argument to do_something()
do_something(mock)

# Patch the json library
json = mock

When you substitute an object in your code, the Mock __must look like__ the _real_ object it is replacing. Otherwise, your code will not be able to use the Mock in place of the original object.

For example, if you are mocking the json library and your program calls `dumps()`, then your Python mock object must also contain `dumps()`.

`Mock` deals with all of this by dynamicaly supporting any function or attribute

# Lazy Attributes and Methods
A Mock must simulate any object that it replaces. 

To achieve such flexibility, it [creates its attributes when you access them](https://docs.python.org/3/library/unittest.mock.html#quick-guide):

In [5]:
mock.some_attribute

<Mock name='mock.some_attribute' id='140629026158912'>

In [6]:
mock.do_something()

<Mock name='mock.do_something()' id='140629025822480'>

Since Mock can create arbitrary attributes on the fly, it is suitable to replace any object.

Using an example from earlier, if you’re mocking the `json` library and you call `dumps()`, the Python mock object will create the method so that its interface can match the library’s interface:



In [7]:
json = Mock()
json.dumps()

<Mock name='mock.dumps()' id='140629025912536'>

Notice two key characteristics of this mocked version of dumps():

1. Unlike the real `dumps()``, this mocked method requires no arguments. In fact, it will accept any arguments that you pass to it.

2. The return value of `dumps()` is also a `Mock`. The capability of `Mock` to recursively define other mocks allows for you to use mocks in complex situations:

In [8]:
json = Mock()
json.loads('{"k": "v"}').get('k')

<Mock name='mock.loads().get()' id='140629026212048'>

Because the return value of each mocked method is also a Mock, you can use your mocks in a multitude of ways.

# Assertions and Inspection
`Mock` instances store data on how you used them. For instance, you can see if you called a method, how you called the method, and so on. There are two main ways to use this information.

First, you can assert that your program used an object as you expected:

In [9]:
# Create a mock object
json = Mock()

json.loads('{"key": "value"}') # <Mock name='mock.loads()' id='4550144184'>

# You know that you called loads() so you can
 # make assertions to test that expectation
json.loads.assert_called()
json.loads.assert_called_once()
json.loads.assert_called_with('{"key": "value"}')
json.loads.assert_called_once_with('{"key": "value"}')

json.loads('{"key": "value"}') # <Mock name='mock.loads()' id='4550144184'>

<Mock name='mock.loads()' id='140629026160536'>

In [14]:
### If an assertion fails, the mock will raise an AssertionError
try:
    json.loads.assert_called_once()
except Exception as ex:
    print(ex)

try:
    json.loads.assert_called_once_with('{"key": "value"}')
except Exception as ex:
    print(ex)

try:
    json.loads.assert_not_called()
except Exception as ex:
    print(ex)

Expected 'loads' to have been called once. Called 2 times.
Expected 'loads' to be called once. Called 2 times.
Expected 'loads' to not have been called. Called 2 times.


### note on `assert_called_with(*args, **kwargs)`

`.assert_called()` ensures you called the mocked method while `.assert_called_once()` checks that you called the method exactly one time.

Both assertion functions have variants that let you inspect the arguments passed to the mocked method:

- `.assert_called_with(*args, **kwargs)`
- `.assert_called_once_with(*args, **kwargs)`

To pass these assertions, you must call the mocked method with __exactly__ the same arguments that you pass to the actual method:

In [17]:
json = Mock()
json.loads(s='{"key": "value"}')
try:
    json.loads.assert_called_with('{"key": "value"}')
except Exception as ex:
    print(ex)

# this works
json.loads.assert_called_with(s='{"key": "value"}')

Expected call: loads('{"key": "value"}')
Actual call: loads(s='{"key": "value"}')


## your turn



In [0]:
class Person:
    def __init__(self, name):
        self.name = name
        self.friends = set()

    def added_as_friend(self, other_person):
        self.friends.add(other_person)

    def add_friend(self, other_person):
        self.added_as_friend(other_person)
        other_person.added_as_friend(self)

In [0]:
def test_add_friend():
    mock = Mock()
    moshe = Person('moshe')
    moshe.add_friend(mock)

    mock.added_as_friend.assert_called_once()

test_add_friend()